In [1]:
import sys
import pandas as pd
import numpy as np
import os
import time
import clr

clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\Thorlabs.MotionControl.DeviceManagerCLI.dll")
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\Thorlabs.MotionControl.GenericMotorCLI.dll")
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\ThorLabs.MotionControl.KCube.DCServoCLI.dll")
from Thorlabs.MotionControl.DeviceManagerCLI import *
from Thorlabs.MotionControl.GenericMotorCLI import *
from Thorlabs.MotionControl.KCube.DCServoCLI import *
from System import Decimal



sys.path.append("../External_Instruments")
sys.path.append("../Multiplexer_Board")
sys.path.append("../Goniometer")



import External_Instruments, PCB_Board, Goniometer


In [2]:
available_setups = External_Instruments.recognize_setup()

available_pins=24
channel_list=[0,1]
current_source=External_Instruments.SourceMeter(baud_rate=57600,name="KEITHLEY 2401", inst_address=available_setups[available_setups["Setup_Name"]=="KEITHLEY 2401"]["GPIB_ADDRESS"].values[0])
voltimeter=current_source

magnet=External_Instruments.Magnetic_Coils(name="KEPCO", inst_address=available_setups[available_setups["Setup_Name"]=="KEPCO"]["GPIB_ADDRESS"].values[0])
#magnet=External_Instruments.Magnetic_Coils(name="HP6032A", inst_address=available_setups[available_setups["Setup_Name"]=="HP6032A"]["GPIB_ADDRESS"].values[0])



multiplexer=PCB_Board.Multiplexer_Board(offline=False,rest_time=0)

#goniometer=Goniometer.Goniometer()
goniometer=False


save_file="B6-24-3"

metadata_dict ={'Batch':3,'Die':3,'Stack':5924,'Annealing_Temperature(Cº)': 300, 'Annealing_Time(h)': 1, 'Annealing_Field(Oe)': 10000,'Annealing_Direction':45,'Thickness(nm)':1.8, "Measurment_Direction": "OOP" ,"Measurment_Angle":0}
#metadata_dict = {'Batch':3,'Die':4,'Stack':5924,'Annealing_Temperature(Cº)': 330, 'Annealing_Time(h)': 1, 'Annealing_Field(Oe)': 0,'Annealing_Direction':0,'Thickness(nm)':1.8, "Measurment_Direction": "IP" ,"Die_angle":'?'}



c:\Users\Pedro\anaconda3\lib\site-packages\pyvisa\ctwrapper\highlevel.py:226: VisaIOWarning: VI_WARN_CONFIG_NLOADED (1073676407): The specified configuration either does not exist or could not be loaded. VISA-specified defaults will be used.
  return self.handle_return_value(session, ret_value)  # type: ignore


['ASRL6::INSTR', 'ASRL8::INSTR', 'ASRL9::INSTR', 'GPIB0::6::INSTR', 'GPIB0::12::INSTR', 'GPIB1::24::INSTR']
KEPCO BOP BIT488 REV. 2.0

NDCI+0.0000E+0,V+1.0000E+0,W+3.0000E-3,L+1.0000E+0

KEITHLEY INSTRUMENTS INC.,MODEL 2401,4026813,A01 Aug 25 2011 12:57:43/A02  /T/K

GPIB1::24::INSTR
:FUNC 'VOLT'
:FORM:ELEM VOLT, CURR, RES, TIME, STAT


c:\Users\Pedro\Desktop\40PinMuxBoard_Modular_V4\Software_INESC\Measurement_Cycles\../External_Instruments\External_Instruments.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_available_setups = df_available_setups.append(new_row,ignore_index=True)


GPIB0::6::INSTR
Found Board!


In [ ]:
multiplexer.set_channel_to_pin(0,40)
multiplexer.set_channel_to_pin(1,1)
current_source.set_current(1e-5)
print(voltimeter.read("VOLT"))


In [5]:
import datetime

def Measurements_available():
    df=pd.DataFrame({ "Measurement_name" : ["2 Probes/Field/Current_Source", "2 Probes/Field/Angular/Current_Source"],
                             "Compatible Current Source"   : [["2K Sourcemeter"], ["2K Sourcemeter"]],
                             "Compatible Voltage Source"   : [["2K Sourcemeter"], ["2K Sourcemeter"]],
                             "Compatible Magnetic Coil": [["400 Oe Coils", "140 Oe Coils", "2K Coils"],["400 Oe Coils", "140 Oe Coils", "2K Coils"]],
                             "Compatible CannonBoard": [["Cannon Board"],["Cannon Board"]],
                             "Compatible Goniometer": [[""],["Goniometer"]],
                             "Compatible Probe Number" : [[2], [2]],
                             "Compatible Measurement Type" : [["Current Source Measurement"],["Current Source Measurement"]]})
    return df

class MeasurmentObject():
    def __init__(self,**kwargs):

        self.measurement_recipe=kwargs.get("measurement_recipe")
        
        self.channel_list=kwargs.get("channel_list")
        self.available_pins=kwargs.get("available_pins")
        self.current_source=kwargs.get("current_source")
        self.voltimeter=kwargs.get("voltimeter")
        self.magnet=kwargs.get("magnet")

        
        self.multiplexer=kwargs.get("pcb_cannon")

        self.pins_with_sensors=kwargs.get("pins_with_sensors")

        self.save_file=kwargs.get("save_file")
        self.goniometer=kwargs.get("goniometer")

        metadata=kwargs.get("metadata")
        self.metadata = metadata if metadata is not None else {}

           
    
    def map_sensor_to_pin(self,current=1e-5,voltage_compliance=10):
        channel1=self.channel_list[0]
        channel2=self.channel_list[1]

        pins_with_sensors=[]
        combination=0

        self.current_source.set_current(current)
        self.voltimeter.set_voltage_compliance(voltage_compliance)
        limit=voltage_compliance*0.95

        for pin1 in range(1,self.available_pins+1):
            self.multiplexer.set_channel_to_pin(channel1,pin1)
            for pin2 in range(1,self.available_pins+1):
                if pin1<=pin2:
                    continue
                
                self.multiplexer.set_channel_to_pin(channel2,pin2)
                measurment_data=self.voltimeter.read("VOLT")
          
                if measurment_data<limit:
                    pins_with_sensors.append([pin1,pin2])
        
        self.pins_with_sensors=pins_with_sensors


        return pins_with_sensors
                    
class Two_Probe_Current_Source(MeasurmentObject):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.H_list=kwargs.get("H_list")
        self.current=kwargs.get("current")
        self.voltage_compliance=kwargs.get("voltage_compliance")
        self.angle_list=kwargs.get("angle_list")
        #self.measurement()
    

    def measurement(self):
        channel1=self.channel_list[0]
        channel2=self.channel_list[1]

        if self.pins_with_sensors==None:
            self.map_sensor_to_pin(current=self.current,voltage_compliance=self.voltage_compliance)

        self.current_source.set_current(self.current)
        self.voltimeter.set_voltage_compliance(self.voltage_compliance)

        Measurement_Data=[]

        for H in self.H_list:
            magnet_current=self.magnet.set_magnetic_field(H)
            #self.current_source.triad(base_frequency=1500,duration=0.05)

            for sensor in self.pins_with_sensors:#
                pin1=sensor[0]
                pin2=sensor[1]
                self.multiplexer.set_channel_to_pin(channel1,pin1)
                self.multiplexer.set_channel_to_pin(channel2,pin2)

                measurment=self.voltimeter.read("VOLT")
                now = datetime.datetime.now()
                Measurement_Data.append([sensor,H, self.current, measurment,measurment/self.current,magnet_current,now])
            
        self.magnet.set_magnetic_field(0)
        

        self.Measurement_Data = pd.DataFrame(Measurement_Data,columns=['Pins','H[Oe]', 'I[A]', 'V[V]', 'R[Ohms]','Magnet Current(A)','Date'])

        for key, value in self.metadata.items():
            self.Measurement_Data[key] = value

        self.Measurement_Data.to_csv(self.save_file + ".csv")
        self.Measurement_Data.to_csv(self.save_file+'.txt', sep='\t', index=False)
        self.multiplexer.close()
                
        if self.magnet.arduino!=False:
             self.magnet.arduino.close()


        #self.current_source.triad(base_frequency=2000,duration=0.5)

        
        return self.Measurement_Data

    
class Two_Probe_Angular_Current_Source(MeasurmentObject):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.H_list=kwargs.get("H_list")
        self.current=kwargs.get("current")
        self.voltage_compliance=kwargs.get("voltage_compliance")
        self.angle_list=kwargs.get("angle_list")
        #self.measurement()

    
    def measurement(self):
        channel1=self.channel_list[0]
        channel2=self.channel_list[1]

        if self.pins_with_sensors==None:
            self.pins_with_sensors = self.map_sensor_to_pin(current=self.current,voltage_compliance=self.voltage_compliance)

        self.current_source.set_current(self.current)
        self.voltimeter.set_voltage_compliance(self.voltage_compliance)

        

        Measurement_Data=[]

        default_accuracy=self.magnet.accuracy

        for H in self.H_list:
            magnet_current=self.magnet.set_magnetic_field(H)
            #self.current_source.triad(base_frequency=1500,duration=0.05)

            for angle in self.angle_list:
                
                if self.goniometer!=False:
                    self.goniometer.set_angle(float(angle))
                    if(angle == self.angle_list[0]):
                        self.magnet.accuracy=default_accuracy
                        magnet_current=self.magnet.set_magnetic_field(H)

                for sensor in self.pins_with_sensors:#
                    pin1=sensor[0]
                    pin2=sensor[1]
                    self.multiplexer.set_channel_to_pin(channel1,pin1)
                    self.multiplexer.set_channel_to_pin(channel2,pin2)

                    measurment=self.voltimeter.read("VOLT")
                    now = datetime.datetime.now()
                    Measurement_Data.append([sensor,H, self.current, measurment,measurment/self.current,magnet_current,angle,now])
            
            if self.goniometer!=False:
                self.magnet.accuracy=100

        self.magnet.set_magnetic_field(0)
        

        self.Measurement_Data = pd.DataFrame(Measurement_Data,columns=['Pins','H[Oe]', 'I[A]', 'V[V]', 'R[Ohms]','Magnet Current(A)','Measurement_Angle','Date'])

        for key, value in self.metadata.items():
            self.Measurement_Data[key] = value

        self.Measurement_Data.to_csv(self.save_file + ".csv")
        #self.Measurment_Data.to_excel(self.save_file + ".xlsx")
        self.Measurement_Data.to_csv(self.save_file+'.txt', sep='\t', index=False)
        
        self.multiplexer.close()
        if self.magnet.arduino!=False:
             self.magnet.arduino.close()

        #self.current_source.triad(base_frequency=2000,duration=0.5)

        
        return self.Measurement_Data

    






             
                



In [3]:

import datetime

class Measurment_Object():
    def __init__(self,channel_list,available_pins,current_source,voltimeter,magnet,multiplexer,save_file,goniometer=False,pins_with_sensors=None,metadata=None):

        self.channel_list=channel_list
        self.available_pins=available_pins
        

        self.current_source=current_source
        self.voltimeter=voltimeter
        self.magnet=magnet

        
        self.multiplexer=multiplexer

        self.pins_with_sensors=pins_with_sensors

        self.save_file=save_file

        self.goniometer=goniometer

        self.metadata = metadata if metadata is not None else {}
        

        #for instrument in external_instruments.ittertuples(index=True, name='Pandas'):
        #    if instrument.Type=="Sourcemeter":
        #        current_source=External_Instruments.SourceMeter(name=instrument.name, address=instrument.adre)
    
    def map_sensor_to_pin(self,current=1e-5,voltage_compliance=10):
        channel1=self.channel_list[0]
        channel2=self.channel_list[1]

        pins_with_sensors=[]
        combination=0

        self.current_source.set_current(current)
        self.voltimeter.set_voltage_compliance(voltage_compliance)
        for pin1 in range(1,self.available_pins+1):
            self.multiplexer.set_channel_to_pin(channel1,pin1)
            for pin2 in range(1,self.available_pins+1):
                if pin1<=pin2:
                    continue
                
                self.multiplexer.set_channel_to_pin(channel2,pin2)
                measurment_data=self.voltimeter.read("VOLT")
          
                if measurment_data<9:
                    pins_with_sensors.append([pin1,pin2])
        
        self.pins_with_sensors=pins_with_sensors
        
        return pins_with_sensors
                    

    def measurment(self,H_list,current,voltage_compliance,angle_list=[0]):
        channel1=self.channel_list[0]
        channel2=self.channel_list[1]

        if self.pins_with_sensors==None:
            self.pins_with_sensors = self.map_sensor_to_pin(current=current,voltage_compliance=voltage_compliance)

        self.current_source.set_current(current)
        self.voltimeter.set_voltage_compliance(voltage_compliance)

        

        Measurement_Data=[]

        #default_accuracy=self.magnet.accuracy

        for H in H_list:
            magnet_current=self.magnet.set_magnetic_field(H)
            self.current_source.triad(base_frequency=1500,duration=0.05)

            for angle in angle_list:
                
                if self.goniometer!=False:
                    self.goniometer.set_angle(float(angle))
                    if(angle == angle_list[0]):
                        self.magnet.accuracy=default_accuracy
                        magnet_current=self.magnet.set_magnetic_field(H)

                for sensor in self.pins_with_sensors:#
                    pin1=sensor[0]
                    pin2=sensor[1]
                    self.multiplexer.set_channel_to_pin(channel1,pin1)
                    self.multiplexer.set_channel_to_pin(channel2,pin2)

                    measurment=self.voltimeter.read("VOLT")
                    now = datetime.datetime.now()
                    Measurement_Data.append([sensor,H, current, measurment,measurment/current,magnet_current,angle,now])
            
            if self.goniometer!=False:
                self.magnet.accuracy=100

        self.magnet.set_magnetic_field(0)
        

        self.Measurement_Data = pd.DataFrame(Measurement_Data,columns=['Pins','H[Oe]', 'I[A]', 'V[V]', 'R[Ohms]','Magnet Current(A)','Measurment_Angle','Date'])

        for key, value in self.metadata.items():
            self.Measurement_Data[key] = value

        self.Measurement_Data.to_csv(self.save_file + ".csv")
        #self.Measurment_Data.to_excel(self.save_file + ".xlsx")
        self.Measurement_Data.to_csv(self.save_file+'.txt', sep='\t', index=False)

        self.current_source.triad(base_frequency=2000,duration=0.5)

        
        return self.Measurement_Data







             
                



In [13]:
measurment_object=Two_Probe_Current_Source(channel_list=channel_list,available_pins=available_pins,current_source=current_source,voltimeter=voltimeter,magnet=magnet,pcb_cannon=multiplexer,save_file=save_file,metadata=metadata_dict,
                                           H_list=H_list,current=1e-5,voltage_compliance=10)

#measurment_object=Measurment_Object(channel_list=channel_list,available_pins=available_pins,current_source=current_source,voltimeter=voltimeter,magnet=magnet,multiplexer=multiplexer,goniometer=goniometer,save_file=save_file,metadata=metadata_dict)



In [14]:
sensors_with_pins=measurment_object.map_sensor_to_pin(current=1e-5)
print(sensors_with_pins)
print(str(len(sensors_with_pins)) + " Sensors found!")


[[2, 1], [4, 3], [6, 5], [8, 7], [10, 9], [12, 11], [16, 15], [18, 17], [20, 19]]
9 Sensors found!


In [15]:
import numpy as np

import numpy as np

H_list = np.concatenate([np.arange(start+(0 if start== -400 or (step <0 and start!=400) else step), end + (step if step > 0 else 0), step) for start, end, step in [(-400, -200, 10), (-200, -100, 4), (-100, 100, 2), (100, 200, 4), (200, 400, 10), (400, 200, -10), (200, 100, -4), (100, -100, -2), (-100, -200, -4) , (-200, -410, -10)]])
#H_list = np.concatenate([np.arange(start+(0 if start== -600 or (step <0 and start!=600) else step), end + (step if step > 0 else 0), step) for start, end, step in [(-1600,-600,100),(-600, -300, 50), (-300, 300, 20), (300, 600, 50), (600,1600,100),(1600,600,-100),(600, 300, -50),(300, -300, -20), (-300, -600, -50),(-600,-1700,100)]])
#angle_list=np.concatenate([np.arange(start+(0 if start== 0 or (step <0 and start!=180) else step), end + (step if step > 0 else 0), step) for start, end, step in [(0,90,10)]])
#print(angle_list)
#print(len(angle_list))
#print(H_list)
print(len(H_list))
#print("Estimated time: " + str(len(H_list)*2.05/60) + "min")


381


In [16]:

#Measurement_Data=measurment_object.measurment(H_list=H_list,current=1e-5,angle_list=angle_list,voltage_compliance=10)
Measurement_Data=measurment_object.measurement()



KeyboardInterrupt: 

In [ ]:
measurment_object.magnet.set_current(0,accuracy=100)